# spec

> Fill in a module description here

In [ ]:
#| default_exp spec

In [ ]:
#| export
from fastcore.all import *
from fastcore.script import *

import httpx, json

In [ ]:
#| hide
import os

In [ ]:
#| exports
stripe_openapi_url = 'https://raw.githubusercontent.com/stripe/openapi/refs/heads/master/openapi/spec3.json'

In [ ]:
stripe_spec = httpx.get(stripe_openapi_url).json()
stripe_spec.keys()

dict_keys(['components', 'info', 'openapi', 'paths', 'security', 'servers'])

The OpenAPI spec describes how a particular REST API works. The most important part of this spec is the paths that are defined by a particular spec. This defines what you can do with a particular API and usually includes things like the description of endpoint, expected parameters, the description of the parameters, and a schema of what will be returned from a particular endpoint.

In [ ]:
p = first(stripe_spec['paths'].items())
p


(
    '/v1/account',
    {
        'get': {
            'description': '<p>Retrieves the details of an account.</p>',
            'operationId': 'GetAccount',
            'parameters': [
                {
                    'description': 'Specifies which fields in the response should be expanded.',
                    'explode': True,
                    'in': 'query',
                    'name': 'expand',
                    'required': False,
                    'schema': {'items': {'maxLength': 5000, 'type': 'string'}, 'type': 'array'},
                    'style': 'deepObject'
                }
            ],
            'requestBody': {
                'content': {
                    'application/x-www-form-urlencoded': {
                        'encoding': {},
                        'schema': {'additionalProperties': False, 'properties': {}, 'type': 'object'}
                    }
                },
                'required': False
            },
            'responses': {


As we can see here, the account path has a single HTTP verb that we can use on it called GET. The verb + path is what we refer to as an endpoint. This endpoint allows us to get the account details for a stripe.

In [ ]:
stripe_api_url = 'https://api.stripe.com'
stripe_api_url + p[0]

'https://api.stripe.com/v1/account'

In [ ]:
stripe_key = os.getenv('STRIPE_SECRET_KEY')
headers = {'Authorization': f'Bearer {stripe_key}'}
resp = httpx.get(stripe_api_url + p[0], headers=headers)
# resp.status_code, resp.json()

Some of these endpoints will take parameters if they are GET verbs or request bodies. POST verbs. Here is an example for the GET customers endpoint

In [ ]:
p = first(stripe_spec['paths'].items(), lambda x: x[0] == '/v1/customers')
p[1]['get']['parameters'][:3]


[
    {
        'description': 'Only return customers that were created during the given date interval.',
        'explode': True,
        'in': 'query',
        'name': 'created',
        'required': False,
        'schema': {
            'anyOf': [
                {
                    'properties': {
                        'gt': {'type': 'integer'},
                        'gte': {'type': 'integer'},
                        'lt': {'type': 'integer'},
                        'lte': {'type': 'integer'}
                    },
                    'title': 'range_query_specs',
                    'type': 'object'
                },
                {'type': 'integer'}
            ]
        },
        'style': 'deepObject'
    },
    {
        'description': "A case-sensitive filter on the list based on the customer's `email` field. The value must be a string.",
        'in': 'query',
        'name': 'email',
        'required': False,
        'schema': {'maxLength': 512, 'type': 'string

And here is one for a post

In [ ]:
list(p[1]['post']['requestBody']['content']['application/x-www-form-urlencoded']['schema']['properties'].items())[:3]


[
    (
        'address',
        {
            'anyOf': [
                {
                    'properties': {
                        'city': {'maxLength': 5000, 'type': 'string'},
                        'country': {'maxLength': 5000, 'type': 'string'},
                        'line1': {'maxLength': 5000, 'type': 'string'},
                        'line2': {'maxLength': 5000, 'type': 'string'},
                        'postal_code': {'maxLength': 5000, 'type': 'string'},
                        'state': {'maxLength': 5000, 'type': 'string'}
                    },
                    'title': 'optional_fields_customer_address',
                    'type': 'object'
                },
                {'enum': [''], 'type': 'string'}
            ],
            'description': "The customer's address."
        }
    ),
    (
        'balance',
        {
            'description': "An integer amount in cents (or local equivalent) that represents the customer's current balance, which aff

Let's make a helper function to grab all these endpoints and their parameters.

In [ ]:
#| export
def stripe_endpoints(spec: dict):
    'Extracts all the endpoints and their parameters from the Stripe OpenAPI spec.'
    endpoints = []
    for path, methods in spec['paths'].items():
        for verb, details in methods.items():
            op_id = details.get('operationId', '')
            summary = details.get('summary', '')
            query_params = [dict(name=p['name'], description=p.get('description', ''))
                            for p in details.get('parameters', []) if p.get('in') == 'query']
            body_params = []
            if 'requestBody' in details:
                schema = nested_idx(details, 'requestBody', 'content', 'application/x-www-form-urlencoded', 'schema', 'properties') or {}
                body_params = [dict(name=k, description=v.get('description', '')) for k,v in schema.items()]
            all_params = query_params + body_params
            endpoints.append(dict(path=path, verb=verb, op_id=op_id, summary=summary, params=all_params))
    return endpoints

In [ ]:
eps = stripe_endpoints(stripe_spec)
eps[0]


{
    'path': '/v1/account',
    'verb': 'get',
    'op_id': 'GetAccount',
    'summary': 'Retrieve account',
    'params': [
        {
            'name': 'expand',
            'description': 'Specifies which fields in the response should be expanded.'
        }
    ]
}

FastStripe follows Stripe's monthly API versioning to ensure stability and compatibility. Rather than automatically using the latest version (which could break existing code when endpoints change), we pin FastStripe releases to specific Stripe API versions.

This approach works by generating a static `endpoints.py` file containing a snapshot of all endpoints, parameters, and documentation from a specific Stripe API version. The FastStripe API then uses these endpoint definitions when interfacing with Stripe.

The helper function below updates both the FastStripe version and the endpoints file to match the latest Stripe API version.

In [ ]:
#| export
@call_parse
def update_version():
    'Update the version to the latest version of the Stripe API and the endpoints file.'
    cfg = Config.find("settings.ini")
    stripe_spec = httpx.get(stripe_openapi_url).json()
    stripe_version = stripe_spec['info']['version'].split('.')[0].replace('-', '.')

    if cfg.d['version'] == stripe_version: return
    cfg.d['version'] = stripe_version
    cfg.save()
    eps = stripe_endpoints(stripe_spec)
    (cfg.config_path/'faststripe/endpoints.py').write_text(f'eps = {json.dumps(eps, indent=4)}')
    print(f'Updated version to {stripe_version}')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()